In [ ]:
import pandas as pd 
import numpy as np
import scipy.stats as ss

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
# set some styling defaults for matplotlib
plt.style.use("seaborn-talk")
mpl.rcParams["figure.dpi"] = 90  # change this to set apparent figure size
mpl.rcParams["figure.figsize"] = (7, 3)
mpl.rcParams["figure.frameon"] = False

# set decimal precision to 3 dec. places
%precision 3

In [ ]:
#%pip install pfilter
#%pip install ipycanvas
# uncomment and run the above if you don't have pfilter and/or ipycanvas installed

In [ ]:
from pfilter import ParticleFilter
from ipycanvas import Canvas


# Example 1: probabilistic filters

## Outcomes
You will understand:
* How probabilistic filtering works
    * How to implement basic probabilistic filters
    * How filtering can be used to extract hidden states
    * How filtering can extend to intention inference
    

# Inference via filtering [30 mins]

## Goal
* Estimate a hidden state in a continuous process from a stream of observations.
* Be able to deal with missing or noisy observations.
* Be able to project forward into the future as needed.
* Be able to quantify uncertainty and the expected value of possible states.

## Process
* We build a **filter**, which in this context means a process that combines observations that occur sequentially.
* Note: it's not really the same as a filter in the sense of signal processing, though the concept of processing sequential signals is the same.
    
* The data generating process is assumed to have some temporal coherence; predictions of the future depend on past states.
* Typically, we make a Markov assumption: that the current unobserved state encodes everything we know about the next state.
    
### Predictor-corrector

The concept is simple: we first build a model that just predicts what we think might be going on -- a pure simulator. Then we can take observations and use them to "filter out" predictions that are unrealistic. This is formulated as a Bayesian belief update.

* Prior at time t + evidence at time t -> posterior at time t

$$P(X_{t+1}|Y_t, X_t) \propto P(Y_t | X_{t+1}) P(X_{t+1})$$
$$P(X_{t+1}|Y_t, X_t) \propto P(Y_t | X_{t+1}) P(X_{t})$$

* (we don't typically care about normalising this distribution, we just want to track *relative* likelihoods of hypotheses)

    

## Package: `pfilter`

We'll use `pfilter` for this

## Setup

In [ ]:
from ipywidgets import Output

out = Output()


@out.capture()
def handle_mouse_move(x, y):
    print("Mouse move event:", x, y)




canvas = Canvas(width=500, height=500)

canvas.fill_rect(25, 25, 100, 100)
canvas.clear_rect(45, 45, 60, 60)
canvas.stroke_rect(50, 50, 50, 50)
canvas.on_mouse_move(handle_mouse_move)
display(canvas)

## Applying

## Reflection